# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [6]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests



# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vung tau,10.3460,107.0843,82.02,71,14,14.03,VN,1675832278
1,1,kapaa,22.0752,-159.3190,75.09,84,17,4.00,US,1675832279
2,2,rabat,33.9911,-6.8401,52.16,81,40,9.22,MA,1675832004
3,3,hobart,-42.8794,147.3294,64.27,63,40,14.97,AU,1675832279
4,4,ushuaia,-54.8000,-68.3000,51.46,71,40,3.44,AR,1675832084


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
# %%capture --no-display

# Configure the map plot
city_data_df = city_data_df.dropna()



map_plot = city_data_df.hvplot.points(
    "Lat",
    "Lng",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
    
)

# Display the map plot
map_plot




:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lat,Lng]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) 
                                    & (city_data_df["Max Temp"] > 70) 
                                    & (city_data_df["Wind Speed"] < 10) 
                                    & (city_data_df["Cloudiness"] == 0),:]

print(f"Number of cities in the list: {len(ideal_weather_df)}")

# Drop any rows with null values
ideal_weather_df=ideal_weather_df.dropna()

print(f"Number of cities in the list: {len(ideal_weather_df)}")
# Display sample data
ideal_weather_df.head()

Number of cities in the list: 11
Number of cities in the list: 11


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,wajir,1.7471,40.0573,75.16,73,0,5.75,KE,1675832286
156,156,dubai,25.2582,55.3047,71.85,40,0,9.22,AE,1675832049
219,219,santiago,-33.4569,-70.6483,76.24,52,0,2.30,CL,1675832326
252,252,moyale,3.5167,39.0584,73.96,33,0,6.40,KE,1675832365
295,295,pisco,-13.7000,-76.2167,73.45,83,0,9.22,PE,1675832380


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng"]].copy()



# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
21,wajir,KE,1.7471,40.0573,
156,dubai,AE,25.2582,55.3047,
219,santiago,CL,-33.4569,-70.6483,
252,moyale,KE,3.5167,39.0584,
295,pisco,PE,-13.7000,-76.2167,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params = {
    "location": f"{lat},{lng}",    
    "radius": 10000,
    "types": "lodging",
    "key": geoapify_key
}


    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = row["City"]
#     params["bias"] = # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"



    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wajir - nearest hotel: No hotel found
dubai - nearest hotel: No hotel found
santiago - nearest hotel: No hotel found
moyale - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
barwani - nearest hotel: No hotel found
lar - nearest hotel: No hotel found
reconquista - nearest hotel: No hotel found
barwadih - nearest hotel: No hotel found
renukut - nearest hotel: No hotel found
salina cruz - nearest hotel: No hotel found


,City,Country,Lat,Lng,Hotel Name
21,wajir,KE,1.7471,40.0573,No hotel found
156,dubai,AE,25.2582,55.3047,No hotel found
219,santiago,CL,-33.4569,-70.6483,No hotel found
252,moyale,KE,3.5167,39.0584,No hotel found
295,pisco,PE,-13.7000,-76.2167,No hotel found
364,barwani,IN,22.0333,74.9000,No hotel found
388,lar,IN,26.2039,83.9717,No hotel found
398,reconquista,AR,-29.1500,-59.6500,No hotel found
420,barwadih,IN,23.8333,84.1167,No hotel found
492,renukut,IN,24.2000,83.0333,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE